In [16]:
from langgraph.graph import StateGraph ,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict ,Literal
from dotenv import load_dotenv
from pydantic import BaseModel ,Field
import os


In [17]:
load_dotenv()

True

In [18]:
api_key = os.getenv("GOOGLE_API_KEY")

In [19]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key   # ✅ use google_api_key, not api_key
)

In [71]:
class SentimentScema(BaseModel):
    sentiment:Literal["Positive","Negative"]= Field(description="sentiment of the review ")

class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')    

In [72]:
prompt ="this is very bad product also its ui and performance it stucks "

structured_model=llm.with_structured_output(SentimentScema)
structured_model2=llm.with_structured_output(DiagnosisSchema)
structured_model2.invoke(prompt)


DiagnosisSchema(issue_type='UX', tone='angry', urgency='high')

In [73]:
class Reviewstate(TypedDict):
    review:str
    sentiment:str
    diagnose:dict
    response:str

In [94]:
def find_sentiment(state:Reviewstate):
    prompt=f'find sentiment ofthe following review \n {state["review"]}'

    sentiment=structured_model.invoke(prompt).sentiment

    return{"sentiment":sentiment}

def check_sentiment(state:Reviewstate) -> Literal["positve_response","run_diagnose"]:
    if state['sentiment']=='positive':
        return "positve_response"
    else:
        return "run_diagnose"
    
def positve_response (state:Reviewstate):

    prompt=f"""write a warm thankful message forh this review \n 
    {state['review']}"""
    response=llm.invoke(prompt).content

    return {'response':response}

def run_diagnose(state:Reviewstate):
    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""  
    response=structured_model2.invoke(prompt)

    
    return {'diagnose': response.model_dump()}
  
def negative_response (state:Reviewstate):
    diagnosis = state['diagnose']

    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.
"""
    response = llm.invoke(prompt).content

    return {'response': response}


In [95]:
graph=StateGraph(Reviewstate)

graph.add_node("find_sentiment",find_sentiment)
graph.add_node ("run_diagnose",run_diagnose)
graph.add_node ("negative_response",negative_response)
graph.add_node ("positve_response",positve_response)


graph.add_edge(START,"find_sentiment")
graph.add_conditional_edges("find_sentiment",check_sentiment)
graph.add_edge("positve_response" ,END)
graph.add_edge("run_diagnose","negative_response")
graph.add_edge("negative_response",END)

workflow=graph.compile()




In [ ]:
workflow

In [96]:
intial_state={
    'review': "I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality."
}
workflow.invoke(intial_state)

{'review': 'I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.',
 'sentiment': 'Negative',
 'diagnose': {'issue_type': 'Bug', 'tone': 'frustrated', 'urgency': 'high'},
 'response': "Subject: Re: Urgent - Bug Report - [Original Subject Line]\n\nHi [User Name],\n\nI'm so sorry to hear you're experiencing this issue and that it's causing you frustration.  I understand how urgent this is, and I want to assure you I'll do everything I can to help resolve it as quickly as possible.\n\nCould you please provide me with a little more information?  Specifically, it would be helpful if you could tell me:\n\n* **What steps were you taking when the bug occurred?**  A detailed description of your actions leading up to the issue will help me reproduce it.\n* **What exactly happened?**  Please describe the bug as clearly as 